# Table of Contents

* [1. Load up libraries](#1.-Load-up-libraries)
    * [1.1 Define functions](#1.1-Define-functions)
* [2. Rebuild database](#2.-Rebuild-database)
    * [2.1 Load data from AWS SQL database](#2.1-Load-data-from-AWS-SQL-database) 
    * [2.2 Build 'freq' and 'normed_pred_diff' columns](#2.2-Build-'freq'-and-'normed_pred_diff'-columns)
        * [2.2.1 Write to database](#2.2.1-Write-to-database)

# 1. Load up libraries

In [1]:
# all purpose
import datetime, re
from math import radians, cos, sin, asin, sqrt

# for talking to SQL databases
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

# all purpose data analysis and plotting
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# for ML
import patsy
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import cross_validation, metrics, linear_model, svm
# needed for cross-validation on sets where the test data is not binary/multiclass 
# (i.e. needed for regressors, not classifiers)
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

# for saving output
import pickle

# load up Muni routes
list_of_muni_routes = np.load('/Users/dstone/Dropbox/insight/project/busUnBunchr_site/list_of_muni_routes.npy')

/Library/Python/2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## 1.1 Define functions

Function that gets frequency for specified route

In [4]:
def get_frequency_for_route(pair_route, pair_time):
    ''' Must past pair_time as a time stamp 
        (which should come from dataframe that you are adding to) 
    '''
    with open('/Users/dstone/Dropbox/insight/project/busUnBunchr_site/route_frequencies/route_'+str(pair_route)+'_frequencies.pkl','rb') as input:
        freq_df  = pickle.load(input)
    t0 = '1970-01-01 '+str(pair_time.hour)+':'+str(pair_time.minute)+':00'
    # set to end of day, no messing around
    t1 = '1970-01-01 23:59:00'
    # some more error catching
    try:
        return freq_df.loc[t0:t1]['freq'][0]
    except:
        # return last frequency you found
        return freq_df.iloc[-1]['freq']

# 2. Rebuild database

## 2.1 Load data from AWS SQL database 

(necessary only to load Muni routes and estlabish connection to database)

We need to connect to the PostgresSQL database that I am reading the NextBus Muni data into, which is called 'sf_muni_arrivals' in our case.

**This cell must be run.**

In [14]:
# MUST BE RUNNING 
# ssh -i ~/.ssh/aws_instance_3_instantiated_2016_02_03.pem -L 63333:127.0.0.1:5432 ec2-user@ec2-52-72-119-113.compute-1.amazonaws.com
# for this to work (for that specific AWS instance)
remote_dbname = 'sf_muni_arrivals_aws'
remote_username = 'ec2-user'
remote_table = 'nextbus_realtime_with_predictions'

# Open up an engine, that we will use to create the database if it doesn't exist
engine = create_engine('postgres://%s@localhost:63333/%s'%(remote_username,remote_dbname))

if not database_exists(engine.url):
    create_database(engine.url)
    
# If I want to filter the data first:
# connect:
db_con = None
db_con = psycopg2.connect(database = remote_dbname, user = remote_username, port = 63333, host = 'localhost')

Now that connection is established, load database into pandas dataframe

In [15]:
sql_query = '''SELECT * FROM {table};'''.format(table=remote_table)
df_all = pd.read_sql_query(sql_query, db_con)

## 2.2 Build 'freq' and 'normed_pred_diff' columns

'freq' is the frequency of the given route at that time of day. For example, during peak hours, the route 30 has 'freq' of 5.0 (it runs every 5 minute).

'normed_pred_diff' is the difference in predictions time compared to the scheduled difference time. For example, if a route runs every 10 minutes and the predictions for the next two vehicles are 2 and 12 (in minutes), 'normed_pred_diff' is abs((2-12)/10) = 1.0. If the predictions were instead 2 and 7, we'd have abs((2-7)/10) = 0.5, suggesting a more bunched bus (note, however, in the final analysis we define a different parameter that time averages this and maps entirely bunched buses (here these would be 'normed_pred_diff' = 0.0) to 1.0).

In [ ]:
# just for cleaning
df_tmp = df_all.drop(['index'], axis=1)
# build 'freq' column
df_tmp['freq'] = df_tmp.apply(lambda row: get_frequency_for_route(row['route_x'],row['time']), axis=1)
# build 'normed_pred_diff' column
df_tmp['normed_pred_diff'] = df_tmp.apply(lambda row: 
                                          (np.abs(float(row['pred_x'])-float(row['pred_y'])))/float(row['freq']), axis=1)

### 2.2.1 Write to database
Now write this back into the database, so we can analyze it later. Use the psychogp2 engine we defined above (this writes back into the database we already connected to). We specify the table as the first argument.

In [19]:
df_tmp.to_sql('nextbus_realtime_with_predictions_and_freqs', engine, if_exists='append')